# **Template OP on salish**

In [56]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels_biofilm import DeleteParticle, Buoyancy, AdvectionRK4_3D, Stokes_drift, Beaching, Unbeaching
from OP_functions_biofilm import *

In [57]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

In [58]:
path_NEMO = make_prefix(datetime(2018, 1, 10), paths['NEMO'])
coord=xr.open_dataset(paths['coords'],decode_times=False)
WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 1, 10)))
batt=xr.open_dataset(paths['mask'],decode_times=False)

## Define and save mask for distance to coast

In [59]:
Dat=xr.open_dataset(paths['coords'],decode_times=False)
outf_lat=Dat['nav_lat'][445,304]
outf_lon=Dat['nav_lon'][445,304]
clon, clat = [float(outf_lon)],[float(outf_lat)]  # choose horizontal centre of the particle cloud
# #clat,clon = p_unidist(coord.gphif[0,:,:],coord.glamf[0,:,:],batt.mbathy[0,:,:],10,10)
# with open('clat.txt') as f:
#     clat = f.read()
#     clat= clat[1:-1]
#     clat0 = clat.split(",")
#     f.close()
# with open('clon.txt') as f:
#     clon = f.read()
#     clon=clon[1:-1]
#     clon0 = clon.split(",")
#     f.close()
    
# clat,clon=[],[]
# for i in range(len(clat0)):
#     clat.append(float(clat0[i]))
#     clon.append(float(clon0[i]))

## Definitions

In [60]:
start = datetime(2018, 6, 11) #Start date
# Set Time length [days] and timestep [seconds]
length = 45
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = 1 # number of deploying locations
n = 500 # 1000   # number of particles per location
dmin = 68 #minimum depth
dd = 5 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [61]:
from parcels import Variable

class MPParticle(JITParticle):        
    ro = Variable('ro', initial = 1025)           
    diameter = Variable('diameter', initial = 1.6e-5)
    length = Variable('length', initial = 61e-5)
    Lb = Variable('Lb', initial = 0.23)  #days needed in days for particle to have 67% probability of beaching if in beaching zone (500m)
    Db = Variable('Db', initial = 33) #Distance at which particles can randomly beach.
    Ub = Variable('Ub', initial = 69)  #days to have 67% probability of unbeaching
    sediment = Variable('sediment', initial = 0)
    beached = Variable('beached', initial = 0)


In [62]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [63]:
#Set start date time and the name of the output file
name = 'f18Summer' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/results/f18Summer20180611_1n_20180726_1n.nc


## Simulation

In [64]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

filenames,variables,dimensions=filename_set(start,length,['Bathy'],local)
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

In [65]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

k_sink = pset.Kernel(Buoyancy)
k_waves = pset.Kernel(Stokes_drift)
k_beach = pset.Kernel(Beaching)
k_unbeach = pset.Kernel(Unbeaching)


pset.execute(AdvectionRK4_3D + k_sink + k_waves + k_beach + k_unbeach,
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DBuoyancyStokes_driftBeachingUnbeaching ==> /tmp/parcels-2894/lib942099f242f053afea274431c37d6609_0.so
INFO: Temporary output files are stored in /home/jvalenti/MOAD/results/out-WVWLZGRQ.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/results/out-WVWLZGRQ" to convert these to a NetCDF file during the run.
N/A% (0 of 3888000.0) |                  | Elapsed Time: 0:00:00 ETA:  --:--:--
  0% (10800.0 of 3888000.0) |            | Elapsed Time: 0:00:00 ETA:   0:01:51
  0% (12600.0 of 3888000.0) |            | Elapsed Time: 0:00:03 ETA:   1:52:54
  0% (14400.0 of 3888000.0) |            | Elapsed Time: 0:00:03 ETA:   1:04:24
  0% (16200.0 of 3888000.0) |            | Elapsed Time: 0:00:07 ETA:   1:53:19
  0% (18000.0 of 3888000.0) |            | Elapsed Time: 0:00:07 ETA:   1:01:57
  0% (19800.0 of 3888000.0) |            | Elapsed Time: 0:00:10 ETA:   1:49:53
  0% (21600.0 of 3888000.0) |            | Elapsed Time: 0:00:10

Correct cell not found for (-124.806612, 48.530939) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 423 0
            new particle indices: (yi, xi) 423 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3013394318500424e+01 4.5220259962544124e-01
Particle 807 lost !! [2564370.0, 10.60382509711414, 48.53101655166488, -124.80597834554625]


 65% (2559600.0 of 3888000.0) |######    | Elapsed Time: 0:40:50 ETA:   0:25:22
 65% (2561400.0 of 3888000.0) |######    | Elapsed Time: 0:40:54 ETA:   0:41:42
 65% (2563200.0 of 3888000.0) |######    | Elapsed Time: 0:40:54 ETA:   0:23:06
 65% (2565000.0 of 3888000.0) |######    | Elapsed Time: 0:40:57 ETA:   0:40:53
 66% (2566800.0 of 3888000.0) |######    | Elapsed Time: 0:40:58 ETA:   0:22:17
 66% (2568600.0 of 3888000.0) |######    | Elapsed Time: 0:41:01 ETA:   0:33:47
 66% (2570400.0 of 3888000.0) |######    | Elapsed Time: 0:41:01 ETA:   0:18:42
 66% (2572200.0 of 3888000.0) |######    | Elapsed Time: 0:41:03 ETA:   0:31:17
 66% (2574000.0 of 3888000.0) |######    | Elapsed Time: 0:41:04 ETA:   0:17:37
 66% (2575800.0 of 3888000.0) |######    | Elapsed Time: 0:41:07 ETA:   0:41:13
 66% (2577600.0 of 3888000.0) |######    | Elapsed Time: 0:41:07 ETA:   0:22:31
 66% (2579400.0 of 3888000.0) |######    | Elapsed Time: 0:41:10 ETA:   0:35:26
 66% (2581200.0 of 3888000.0) |######   

Correct cell not found for (-124.787188, 48.507803) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 417 0
            new particle indices: (yi, xi) 417 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.2975897845676030e+01 6.4764922351092469e-01
Particle 854 lost !! [2817000.0, 4.148823288945351, 48.507825163881535, -124.78644306253345]


 72% (2813400.0 of 3888000.0) |#######   | Elapsed Time: 0:44:54 ETA:   0:31:09
 72% (2815200.0 of 3888000.0) |#######   | Elapsed Time: 0:44:55 ETA:   0:17:23
 72% (2817000.0 of 3888000.0) |#######   | Elapsed Time: 0:44:58 ETA:   0:30:11
 72% (2818800.0 of 3888000.0) |#######   | Elapsed Time: 0:44:58 ETA:   0:16:31
 72% (2820600.0 of 3888000.0) |#######   | Elapsed Time: 0:45:01 ETA:   0:27:32
 72% (2822400.0 of 3888000.0) |#######   | Elapsed Time: 0:45:01 ETA:   0:15:25
 72% (2824200.0 of 3888000.0) |#######   | Elapsed Time: 0:45:04 ETA:   0:25:33
 72% (2826000.0 of 3888000.0) |#######   | Elapsed Time: 0:45:04 ETA:   0:14:24
 72% (2827800.0 of 3888000.0) |#######   | Elapsed Time: 0:45:08 ETA:   0:32:59
 72% (2829600.0 of 3888000.0) |#######   | Elapsed Time: 0:45:08 ETA:   0:18:16
 72% (2831400.0 of 3888000.0) |#######   | Elapsed Time: 0:45:11 ETA:   0:32:38
 72% (2833200.0 of 3888000.0) |#######   | Elapsed Time: 0:45:12 ETA:   0:18:07
 72% (2835000.0 of 3888000.0) |#######  

Correct cell not found for (-124.809363, 48.534230) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 424 0
            new particle indices: (yi, xi) 424 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3023590520563559e+01 2.8648014021602369e-01
Particle 872 lost !! [3306780.0, 14.360654820943694, 48.53442047574677, -124.80879651135864]


 84% (3303000.0 of 3888000.0) |########  | Elapsed Time: 0:52:45 ETA:   0:20:59
 85% (3304800.0 of 3888000.0) |########  | Elapsed Time: 0:52:45 ETA:   0:11:29
 85% (3306600.0 of 3888000.0) |########  | Elapsed Time: 0:52:48 ETA:   0:15:24
 85% (3308400.0 of 3888000.0) |########  | Elapsed Time: 0:52:48 ETA:   0:08:28
 85% (3310200.0 of 3888000.0) |########  | Elapsed Time: 0:52:51 ETA:   0:13:29
 85% (3312000.0 of 3888000.0) |########  | Elapsed Time: 0:52:51 ETA:   0:07:32
 85% (3313800.0 of 3888000.0) |########  | Elapsed Time: 0:52:54 ETA:   0:14:11
 85% (3315600.0 of 3888000.0) |########  | Elapsed Time: 0:52:54 ETA:   0:07:53
 85% (3317400.0 of 3888000.0) |########  | Elapsed Time: 0:52:58 ETA:   0:18:59
 85% (3319200.0 of 3888000.0) |########  | Elapsed Time: 0:52:58 ETA:   0:10:27
 85% (3321000.0 of 3888000.0) |########  | Elapsed Time: 0:53:01 ETA:   0:17:28
 85% (3322800.0 of 3888000.0) |########  | Elapsed Time: 0:53:02 ETA:   0:09:41
 85% (3324600.0 of 3888000.0) |######## 